In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

In [17]:
df = pd.read_csv('titanic/train.csv')
train = df.sample(frac=0.7)
test = df.drop(df.sample(frac=0.7).index)
# train_index = np.arange(df.shape[0])

In [41]:
label = 'Survived'
save_path = 'agModels-predictClass'
predictor = TabularPredictor(label=label, path=save_path).fit(train,presets='best_quality')

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=6, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass\"
AutoGluon Version:  0.7.0
Python Version:     3.8.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
Train Data Rows:    624
Train Data Columns: 11
Label Column: Survived
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    17474.58 MB
	Train Data (Or

In [42]:
predictor.leaderboard(train, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,1.000000,0.631410,0.023000,0.019001,0.009000,0.023000,0.019001,0.009000,1,True,2
1,RandomForestEntr_BAG_L1,1.000000,0.822115,0.107005,0.083999,0.729000,0.107005,0.083999,0.729000,1,True,6
2,RandomForestGini_BAG_L1,1.000000,0.815705,0.109000,0.152000,0.716999,0.109000,0.152000,0.716999,1,True,5
3,ExtraTreesGini_BAG_L1,1.000000,0.820513,0.109779,0.148998,0.618997,0.109779,0.148998,0.618997,1,True,8
4,ExtraTreesEntr_BAG_L1,1.000000,0.817308,0.116407,0.100002,0.683124,0.116407,0.100002,0.683124,1,True,9
5,LightGBMLarge_BAG_L1,0.974359,0.846154,0.073999,0.033006,1.716830,0.073999,0.033006,1.716830,1,True,13
6,XGBoost_BAG_L1,0.972756,0.838141,0.143000,0.039999,1.336674,0.143000,0.039999,1.336674,1,True,11
7,LightGBM_BAG_L1,0.955128,0.854167,0.044000,0.033998,1.030998,0.044000,0.033998,1.030998,1,True,4
8,WeightedEnsemble_L2,0.955128,0.854167,0.047000,0.036020,1.860472,0.003000,0.002022,0.829474,2,True,14
9,NeuralNetTorch_BAG_L1,0.929487,0.854167,0.142001,0.140996,8.688399,0.142001,0.140996,8.688399,1,True,12


In [35]:
y_test = test[label] 
test_data_nolab = test.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.9063670411985019
Evaluations on test data:
{
    "accuracy": 0.9063670411985019,
    "balanced_accuracy": 0.8903508771929824,
    "mcc": 0.7947194142390263,
    "f1": 0.8648648648648649,
    "precision": 0.898876404494382,
    "recall": 0.8333333333333334
}


Predictions:  
 0      0
2      0
5      0
7      0
12     0
      ..
855    1
866    1
867    0
869    1
876    0
Name: Survived, Length: 267, dtype: int64


In [43]:
predictor.leaderboard(test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTreesEntr_BAG_L1,0.936330,0.817308,0.105995,0.100002,0.683124,0.105995,0.100002,0.683124,1,True,9
1,RandomForestGini_BAG_L1,0.928839,0.815705,0.092220,0.152000,0.716999,0.092220,0.152000,0.716999,1,True,5
2,ExtraTreesGini_BAG_L1,0.928839,0.820513,0.105004,0.148998,0.618997,0.105004,0.148998,0.618997,1,True,8
3,RandomForestEntr_BAG_L1,0.917603,0.822115,0.099715,0.083999,0.729000,0.099715,0.083999,0.729000,1,True,6
4,XGBoost_BAG_L1,0.917603,0.838141,0.133996,0.039999,1.336674,0.133996,0.039999,1.336674,1,True,11
5,LightGBMLarge_BAG_L1,0.913858,0.846154,0.066998,0.033006,1.716830,0.066998,0.033006,1.716830,1,True,13
6,LightGBM_BAG_L1,0.906367,0.854167,0.038002,0.033998,1.030998,0.038002,0.033998,1.030998,1,True,4
7,WeightedEnsemble_L2,0.906367,0.854167,0.040002,0.036020,1.860472,0.002000,0.002022,0.829474,2,True,14
8,NeuralNetTorch_BAG_L1,0.895131,0.854167,0.122002,0.140996,8.688399,0.122002,0.140996,8.688399,1,True,12
9,KNeighborsDist_BAG_L1,0.883895,0.631410,0.022000,0.019001,0.009000,0.022000,0.019001,0.009000,1,True,2


In [36]:
pre_data = pd.read_csv('titanic/test.csv')

In [37]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(pre_data)

In [39]:
result = pd.DataFrame(pre_data['PassengerId'])
result['Survived'] = y_pred

In [40]:
result.to_csv('titanic/submission.csv',index=False)